In [ ]:
import numpy as np

def simulate_var(B, c, e, n):
    """
    Simulates a VAR model.
    
    B: Coefficient matrix. Shape (k, maxlags, k). B[i, j, l] is the coefficient for the lth variable at lag j in the ith equation.
    c: Constants vector. Length k. c[i] is the constant term for the ith equation.
    e: Error terms. Shape (k, n). e[i, t] is the error term for the ith equation at time t.
    n: Number of observations to simulate.
    """
    
    maxlags = B.shape[1]
    k = B.shape[0]

    # Initialize the series
    Y = np.zeros((k, n))

    # Fill in the first few values with random errors
    for t in range(maxlags):
        for i in range(k):
            Y[i, t] = e[i, t]

    # Main loop
    for t in range(maxlags, n):
        for i in range(k):
            # Start with the constant term
            Y[i, t] = c[i]

            # Add the lagged terms
            for j in range(maxlags):
                for l in range(k):
                    Y[i, t] += B[i, j, l] * Y[l, t-j-1]

            # Add the error term
            Y[i, t] += e[i, t]

    return Y
